<a href="https://colab.research.google.com/github/anniechen0506/Modeling-and-Optimization-Fall-2023/blob/main/Optimization_with_for_Loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set-Up

In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

In [ ]:
import pandas as pd

#Your Turn Looping

Find the minimum of the function $f(x) = ax^2 - bx + c$

For the 100 problems with different instances of a, b, c, given in the dataset.

Below, I've started the problem by solving just one instance.

In [ ]:
#use pandas to import the data
df = pd.read_excel("YTLoopData.xlsx", sheet_name = "data")
df

,Problem_Number,a,b,c
0,1,4,15,34
1,2,2,9,10
2,3,1,12,33
3,4,5,3,64
4,5,4,6,19
...,...,...,...,...
95,96,5,5,78
96,97,3,15,73
97,98,3,5,100
98,99,5,10,57


In [ ]:
def solve_model(a,b,c):
  #initialize a "Concrete Model"
  model = ConcreteModel()

  #initialize DVs
  model.x = Var(domain = Reals)

  #define the objective
  model.obj = Objective(expr = a*model.x**2 - b*model.x + c, sense = minimize)

  #solve model
  opt = SolverFactory('ipopt')
  results = opt.solve(model, tee = False)

  return {"x*": model.x(), "obj*": model.obj()}

In [ ]:
solve_model(100,3,4)

{'x*': 0.015, 'obj*': 3.9775}

In [ ]:
row_data = df.iloc[0].tolist() #first row
a = row_data[1]
b = row_data[2]
c = row_data[3]
solve_model(a,b,c)

{'x*': 1.875, 'obj*': 19.9375}

In [ ]:
solutions = []
for i in range(len(df)):
  row_data = df.iloc[i].tolist()
  a = row_data[1]
  b = row_data[2]
  c = row_data[3]
  solutions.append(solve_model(a,b,c))

In [ ]:
df["solutions"] = solutions
df

,Problem_Number,a,b,c,solutions
0,1,4,15,34,"{'x*': 1.875, 'obj*': 19.9375}"
1,2,2,9,10,"{'x*': 2.25, 'obj*': -0.125}"
2,3,1,12,33,"{'x*': 6.0, 'obj*': -3.0}"
3,4,5,3,64,"{'x*': 0.3, 'obj*': 63.55}"
4,5,4,6,19,"{'x*': 0.75, 'obj*': 16.75}"
...,...,...,...,...,...
95,96,5,5,78,"{'x*': 0.5, 'obj*': 76.75}"
96,97,3,15,73,"{'x*': 2.5, 'obj*': 54.25}"
97,98,3,5,100,"{'x*': 0.8333333333333333, 'obj*': 97.91666666..."
98,99,5,10,57,"{'x*': 1.0, 'obj*': 52.0}"
